# Week 3 assignment - Toronto clustering part 1.

## Task 1 - Transforming wiki data into a pandas data frame

In [58]:
#importing necessary packages
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np

In [48]:
# obtaining the url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikidata = requests.get(url).text
transdata = BeautifulSoup(wikidata, 'html.parser')

### cleaning up the data and conversion of data into a list


In [49]:
data = []
for tr in transdata.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.head(10)

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


In [50]:
# lets observe the dimension of data
df.shape

(289, 3)

In [51]:
# removing non assigned rows in borough and neighbourhood
NA = 'Not assigned'
Na_rows = df[(df.Borough == NA)&(df.Neighborhood == NA)]
print(Na_rows.shape)
Na_rows.head(10)

(77, 3)


,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
10,M8A,Not assigned,Not assigned
14,M2B,Not assigned,Not assigned
21,M7B,Not assigned,Not assigned
22,M8B,Not assigned,Not assigned
31,M2C,Not assigned,Not assigned
37,M7C,Not assigned,Not assigned
38,M8C,Not assigned,Not assigned
46,M2E,Not assigned,Not assigned


It is observed that we have 77 rows where both Borough and neighborhood are not assigned .We need to remove this data from the table.
Note that where only neighborhood has not assigned values , it will have to be replaced with the borough value.

In [52]:
# dropping the unnecessary data
df.drop(Na_rows.index, inplace=True)
# dropping none value aswell
df.dropna(inplace=True)
print("dropped successfully")
#reset index
df=df.reset_index(drop=True)


dropped successfully


In [53]:
print(df.head(10))
print(df.shape)

  PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights
5        M6A        North York    Lawrence Manor
6        M7A      Queen's Park      Not assigned
7        M9A         Etobicoke  Islington Avenue
8        M1B       Scarborough             Rouge
9        M1B       Scarborough           Malvern
(211, 3)


## Task 2:
### Rearranging the neighorhood column .
* All non assigned neighborhoods need to be assigned with their respective borough values

In [65]:
#replacing neighborhoods with borough
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
print(df.head(10)) # note how queenspark has been replaced with queenspark in neighbourhood which had a prior value of not assigned
df.tail()

  PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights
5        M6A        North York    Lawrence Manor
6        M7A      Queen's Park      Queen's Park
7        M9A         Etobicoke  Islington Avenue
8        M1B       Scarborough             Rouge
9        M1B       Scarborough           Malvern


,PostalCode,Borough,Neighborhood
206,M8Z,Etobicoke,Kingsway Park South West
207,M8Z,Etobicoke,Mimico NW
208,M8Z,Etobicoke,The Queensway West
209,M8Z,Etobicoke,Royal York South West
210,M8Z,Etobicoke,South of Bloor


# Task 3:
## Consolidating dataframe with unique potal codes and reassigning all neighburhoods which fall in the same region

In [60]:
# number of unique postal codes , boroughs and neighbourhoods
postalcodes = df['PostalCode'].nunique()
boroughs = df['Borough'].nunique()
neighborhood = df['Neighborhood'].nunique()
print("the unique values of postalcode , boroughs and neighbourhood respectively are : " ,postalcodes ,",",boroughs,",",neighborhood)


the unique values of postalcode , boroughs and neighbourhood respectively are :  103 , 11 , 209


__Aggregating neighbourhood__

In [62]:
# let us clone the dataframe
df1 = df
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [63]:
nrows1 = len(df1)
print(nrows1)  # total number of rows
nrows2 = len(df1)-1
print(nrows2) # total number of rows excluding header

#using two different indexings 

211
210


In [71]:
n=0

while n < nrows2 :
    postabove=df1.iloc[n,0]
    #post1
    m=n+1
    postbelow=df1.iloc[m,0]
    #post2
    neigh1=df1.iloc[n,2]
    neigh2=df1.iloc[m,2]
    if postabove==postbelow:       # if the postal code above and below are the same merge them into one
        df1.Neighborhood[n,2] = neigh1=neigh1+','+neigh2
        
        df1=df1.drop(df1.index[m])
        nrows2=nrows2-1
        df1 = df1.reset_index(drop=True)
    else:
        n=n+1


df1 = df1.reset_index(drop=True)

df1.index

RangeIndex(start=0, stop=103, step=1)

In [73]:
df1.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw..."
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [75]:
df1.tail()


,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
102,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."


### successfully merged unique postal codes and respective neighbourhoods

In [76]:
df1.head(30)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw..."
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [78]:
df1.shape

(103, 3)

### end of part 1